In [4]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import numpy as np
import networkx as nx
import funcy as fy
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt

import usgoc.preprocessing.graph.wl2 as wl2
import usgoc.datasets.unsafe_go as dataset
import usgoc.models.gnn as gnn
import usgoc.utils as utils

In [5]:
split_i = 0 # the evaluated split number (between 0-9)
in_enc = "wl1"

with utils.cache_env(use_cache=True):
  ds = dataset.load_dataset()
  splits = dataset.get_split_idxs(ds)

In [3]:
from transformers import CodeGenTokenizer

checkpoint = "Salesforce/codegen-350M-multi"
tokenizer = CodeGenTokenizer.from_pretrained(checkpoint, cache_dir="/app/.cache")
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
tokenizer

Using pad_token, but it is not set yet.


PreTrainedTokenizer(name_or_path='Salesforce/codegen-350M-multi', vocab_size=50257, model_max_len=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|pad|>'})

In [5]:
import gc
import torch
from transformers import CodeGenModel, CodeGenForCausalLM

gc.collect()
torch.cuda.empty_cache()

max_memory_mapping = {0: "3000MB"}
model = CodeGenModel.from_pretrained(
  checkpoint,
  device_map="auto", load_in_8bit=True,
  max_memory=max_memory_mapping
)

Some weights of the model checkpoint at Salesforce/codegen-350M-multi were not used when initializing CodeGenModel: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing CodeGenModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CodeGenModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
graphs = ds[0]
codes = [g.source_code for g in graphs]

In [8]:
import json

with open("snippets.json", "w") as f:
  json.dump(codes, f)

In [7]:
embs = []
for code_part in fy.partition(10, codes):
  print(len(code_part[0]))
  gc.collect()
  torch.cuda.empty_cache()
  tokens = tokenizer(
    code_part, return_tensors="pt", padding=True, truncation=True, 
    max_length=2048)
  with torch.no_grad():
    model(**tokens)

111


: 

: 

In [13]:
torch.cuda.empty_cache()

In [8]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 1            |        cudaMalloc retries: 1         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |    6309 MB |    6309 MB |    6321 MB |   11744 KB |
|       from large pool |    6309 MB |    6309 MB |    6320 MB |   11744 KB |
|       from small pool |       0 MB |       0 MB |       0 MB |       0 KB |
|---------------------------------------------------------------------------|
| Active memory         |    6309 MB |    6309 MB |    6321 MB |   11744 KB |
|       from large pool |    6309 MB |    6309 MB |    6320 MB |